In [1]:
from skimage import measure

In [2]:
import cv2
import dicom
from keras.layers import LSTM, Dense, Dropout, TimeDistributed
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import math
from matplotlib import path
from matplotlib import pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.misc import imresize
from skimage.measure import find_contours
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

Using TensorFlow backend.


In [3]:
IMAGE_SIZE = 512
NB_EPOCH = 1000
AUG_CONTOUR_NUM = 10
AUG_TRANSFORM_NUM = 32

In [4]:
X = []
Y = []

masks_prefix = "/home/a.kondyukov/data/Indianapolis_masks_new/"
cases = ["atherosclerosis", "calcinosis", "cardiomegaly_mild", "cardiomegaly_modsev", "normal", "tortuous_aorta"]

for i, case in enumerate(cases):
    filenames = !ls /home/a.kondyukov/data/Indianapolis_masks_new/"$case""_masks" | grep "\.png$"
    
    print(case)
    print("\r", "0%", end="")
    
    max_iter = len(filenames)
    for j, filename in enumerate(filenames):
        print("\r", "%.2f" % (j / max_iter * 100) + "%", end="")
        ref_img = plt.imread(os.path.join(masks_prefix, case + "_masks", filename))
        cur_img = imresize(ref_img, [IMAGE_SIZE, IMAGE_SIZE])[:, :, :3].mean(axis=2)
        X.append(cur_img)
        Y.append(i)
    print()
        
X = np.array(X)
Y = np.array(Y)

X /= X.max()

atherosclerosis
 98.61%
calcinosis
 97.67%
cardiomegaly_mild
 98.18%
cardiomegaly_modsev
 96.00%
normal
 98.67%
tortuous_aorta
 98.28%


In [5]:
contours = []
labels = []

f = np.vectorize(math.atan)

datagen = ImageDataGenerator(
                    featurewise_center=False,
                    featurewise_std_normalization=False,
                    rotation_range=10.,
                    zoom_range=0.1,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    fill_mode='constant',
                    cval=0.,
                    horizontal_flip=False)

print("\r", "0%", end="")
max_iter = len(X)

eps = 1e-7

for counter, (mask, label) in enumerate(zip(X, Y)):
    print("\r", "%.2f" % (counter / max_iter * 100) + "%", end="")
    for trans_counter, mask_aug in enumerate(datagen.flow(mask[None, None, :, :], batch_size=1)):
        if trans_counter > AUG_TRANSFORM_NUM:
            break
        
        cur_contours = find_contours(mask_aug[0, 0], 0.1)

        areas = []
        for i, contour in enumerate(cur_contours):
            area = cv2.contourArea(contour[:, None, :].astype(int))
            areas.append(area)

        idx = np.argsort(areas)[:-3:-1]
        contours_filt = [cur_contours[i] for i in range(len(cur_contours)) if i in idx]
        if contours_filt[0][:, 1].max() < contours_filt[1][:, 1].max():
            contours_filt = contours_filt[::-1]


        for aug in range(AUG_CONTOUR_NUM):
            rand_idx = np.random.choice(np.arange(contours_filt[0].shape[0]), 200, replace=False)
            rand_idx.sort()
            arr_left = contours_filt[0][rand_idx]
            arr_left = arr_left[1:, :] - arr_left[:-1, :]
    #         arr_left = f(arr_left[:, 0] / arr_left[:, 1])
            arr_left = np.vstack([
                f(arr_left[:, 0] / (arr_left[:, 1] + eps)),
                np.sqrt(np.square(arr_left[:, 0]) + np.square(arr_left[:, 1]))
                ]).T

            rand_idx = np.random.choice(np.arange(contours_filt[1].shape[0]), 200, replace=False)
            rand_idx.sort()
            arr_right = contours_filt[1][rand_idx]
            arr_right = arr_right[1:, :] - arr_right[:-1, :]
    #         arr_right = f(arr_right[:, 0] / arr_right[:, 1])
            arr_right = np.vstack([
                f(arr_right[:, 0] / (arr_right[:, 1] + eps)),
                np.sqrt(np.square(arr_right[:, 0]) + np.square(arr_right[:, 1]))
                ]).T

            contours.append([arr_left, arr_right])
            labels.append(label)

 99.70%

In [6]:
contours = np.array(contours)
labels = pd.get_dummies(np.array(labels)).values

In [20]:
train_idx, validate_idx = train_test_split(np.arange(len(contours)))

np.savez_compressed("train_test_split", train_idx=train_idx, validate_idx=validate_idx)
X_train, X_validate = contours[train_idx], contours[validate_idx]
Y_train, Y_validate = labels[train_idx], labels[validate_idx]

In [21]:
np.savez_compressed("tmpXY", X_train=X_train, X_validate=X_validate, Y_train=Y_train, Y_validate=Y_validate)

In [ ]:
XY_file = np.load("tmpXY.npz")
X_train = XY_file["X_train"]
X_validate = XY_file["X_validate"]
Y_train = XY_file["Y_train"]
X_validate = XY_file["X_validate"]

In [22]:
model = Sequential()
# model.add(Dense(1024, activation="relu", input_shape=(796, )))
# model.add(Dropout(0.4))
# model.add(Dense(1024, activation="relu"))
model.add(LSTM(1024, return_sequences=True, input_shape=(398, 2)))
model.add(Dropout(0.2))
model.add(LSTM(1024))
model.add(Dropout(0.2))
# model.add(LSTM(1024))
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1024, activation="relu"))
model.add(Dropout(0.2))
# model.add(Dropout(0.2))
model.add(Dense(6, activation="softmax"))

In [23]:
model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
sc = MinMaxScaler()

# X_train = X_train.reshape(X_train.shape[0], -1)
# X_validate = X_validate.reshape(X_validate.shape[0], -1)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1] * X_train.shape[2], X_train.shape[3])
X_validate = X_validate.reshape(X_validate.shape[0], X_validate.shape[1] * X_validate.shape[2], X_validate.shape[3])

ref_shape_train = X_train.shape
ref_shape_validate = X_validate.shape

X_train = sc.fit_transform(X_train.reshape([ref_shape_train[0], -1])).reshape(ref_shape_train)
X_validate = sc.transform(X_validate.reshape([ref_shape_validate[0], -1])).reshape(ref_shape_validate)

In [ ]:
model.fit(X_train, Y_train, 
          batch_size=128, 
          nb_epoch=NB_EPOCH, 
          validation_data=(X_validate, Y_validate))

Train on 81180 samples, validate on 27060 samples
Epoch 1/1000
 2176/81180 [..............................] - ETA: 3139s - loss: 1.8596 - acc: 0.2063

In [ ]:
pred = model.predict_classes(sc.transform(X_validate.reshape(-1, 198)))

In [ ]:
t = np.argmax(Y_validate, axis=1)

In [ ]:
(t == pred).mean()